<a href="https://colab.research.google.com/github/AJ112103/IML_Assignment_2A/blob/main/OneConvMaxPool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        (0.4914, 0.4822, 0.4465),
        (0.2470, 0.2435, 0.2616)
    ),
])
trainloader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('./data', train=True,  download=True, transform=transform),
    batch_size=128, shuffle=True, num_workers=4
)
testloader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('./data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4914,0.4822,0.4465),
                                                         (0.2470,0.2435,0.2616))
                                ])),
    batch_size=256, shuffle=False, num_workers=2
)


In [11]:
class ConvPoolCIFAR(nn.Module):
    def __init__(self, num_filters=128, kernel_size=5, pool_size=4):
        super().__init__()
        self.conv1   = nn.Conv2d(3, num_filters//2, kernel_size, padding=kernel_size//2)
        self.conv2   = nn.Conv2d(num_filters//2, num_filters, kernel_size, padding=kernel_size//2)
        self.relu    = nn.ReLU(inplace=True)
        self.pool    = nn.MaxPool2d(pool_size)
        self.dropout = nn.Dropout(0.3)
        fc_in = num_filters * (32 // pool_size) ** 2
        self.fc      = nn.Linear(fc_in, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)

model = ConvPoolCIFAR(num_filters=128, kernel_size=3, pool_size=4).to(device)

In [12]:
lr = 0.05
epochs = 60
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=lr,
    momentum=0.9,
    weight_decay=1e-4
)
scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=20,
    gamma=0.5
)

In [13]:
best_test = 0.0
for epoch in range(1, epochs+1):
    # train
    model.train()
    correct = total = 0
    for x,y in trainloader:
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        preds = out.argmax(dim=1)
        correct += (preds==y).sum().item()
        total   += y.size(0)
    train_acc = correct/total

    # eval
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x,y in testloader:
            x,y = x.to(device), y.to(device)
            preds = model(x).argmax(dim=1)
            correct += (preds==y).sum().item()
            total   += y.size(0)
    test_acc = correct/total
    best_test = max(best_test, test_acc)

    scheduler.step()
    print(f"Epoch {epoch}/{epochs} — "
          f"Train {train_acc*100:.2f}%, "
          f"Test {test_acc*100:.2f}%, "
          f"Best {best_test*100:.2f}%")

Epoch 1/60 — Train 39.36%, Test 53.19%, Best 53.19%
Epoch 2/60 — Train 48.13%, Test 57.08%, Best 57.08%
Epoch 3/60 — Train 50.44%, Test 59.02%, Best 59.02%
Epoch 4/60 — Train 52.21%, Test 61.07%, Best 61.07%
Epoch 5/60 — Train 53.51%, Test 61.23%, Best 61.23%
Epoch 6/60 — Train 54.37%, Test 64.78%, Best 64.78%
Epoch 7/60 — Train 55.25%, Test 64.61%, Best 64.78%
Epoch 8/60 — Train 55.79%, Test 63.75%, Best 64.78%
Epoch 9/60 — Train 57.00%, Test 64.58%, Best 64.78%
Epoch 10/60 — Train 56.91%, Test 67.16%, Best 67.16%
Epoch 11/60 — Train 57.63%, Test 65.66%, Best 67.16%
Epoch 12/60 — Train 57.55%, Test 66.22%, Best 67.16%
Epoch 13/60 — Train 58.37%, Test 67.15%, Best 67.16%
Epoch 14/60 — Train 58.70%, Test 67.79%, Best 67.79%
Epoch 15/60 — Train 58.87%, Test 64.81%, Best 67.79%
Epoch 16/60 — Train 59.48%, Test 69.38%, Best 69.38%
Epoch 17/60 — Train 59.43%, Test 67.95%, Best 69.38%
Epoch 18/60 — Train 59.44%, Test 68.41%, Best 69.38%
Epoch 19/60 — Train 59.40%, Test 69.17%, Best 69.38%
Ep